#### CSC 180 Intelligent Systems 

#### William Lorence

#### California State University, Sacramento


# Project 2: Modern Low Footprint Cyber Attack Protection
## Reading the Data
The code below reads the data from the dataset and creates dataframes. Values of "-" are treated as N/A and entries with this value are dropped from the dataframe. The "attack_cat" column has been dropped to decrease the likelihood of model overfitting. 

In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

path = "./dataset/"
save_path = "./models/"

#Defines filepaths for the data sets
training_set = os.path.join(path,"UNSW_NB15_training-set.csv")
test_set = os.path.join(path,"UNSW_NB15_test-set.csv")

#Loads files into dataframes
df_training_set = pd.read_csv(training_set, na_values = ['-'])
df_test_set = pd.read_csv(test_set, na_values = ['-'])

#Removes rows with a "-" in any column
df_training_set.dropna(inplace = True)
df_test_set.dropna(inplace = True)

#Drop the "attack_cat" column from both the training and test datasets
df_training_set.drop(columns=['attack_cat'], inplace = True)
df_test_set.drop(columns=['attack_cat'], inplace = True)

print(len(df_training_set))
print(len(df_test_set))

df_training_set.head()


81173
35179


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,1,3,1,1,0,2,1,0,0
11,12,2.093085,tcp,smtp,FIN,62,28,56329,2212,42.520967,...,1,1,2,0,0,0,1,1,0,0
15,16,0.000002,udp,snmp,INT,2,0,138,0,500000.001300,...,1,1,4,0,0,0,2,1,0,0
17,18,0.393556,tcp,http,FIN,10,8,860,1096,43.195886,...,1,1,2,0,0,1,1,3,0,0
21,22,0.338017,tcp,http,FIN,10,6,998,268,44.376468,...,1,1,1,0,0,1,2,3,0,0


## Data Filtering
The code below removes categorical values that are not present in both datasets. As visible via the print statements, roughly 600 entries are dropped from the training set, while only one is dropped from the test set.

In [20]:
#Removes categorical values not present in both datasets
categorical_columns = ['proto', 'service', 'state']

for column in categorical_columns:
    unique_values_training = set(df_training_set[column].unique())
    unique_values_test = set(df_test_set[column].unique())

    print(unique_values_training)
    print(unique_values_test)
    
    common_values = unique_values_training.intersection(unique_values_test)
    print(common_values)

    df_training_set = df_training_set[df_training_set[column].isin(common_values)]
    df_test_set = df_test_set[df_test_set[column].isin(common_values)]

print(len(df_training_set))
print(len(df_test_set))

{'udp', 'tcp'}
{'udp', 'tcp'}
{'udp', 'tcp'}
{'radius', 'http', 'smtp', 'dhcp', 'ftp', 'ssh', 'ssl', 'pop3', 'dns', 'ftp-data', 'snmp', 'irc'}
{'radius', 'smtp', 'dhcp', 'ftp', 'ssh', 'ssl', 'snmp', 'pop3', 'irc', 'http', 'dns', 'ftp-data'}
{'radius', 'http', 'smtp', 'dhcp', 'ftp-data', 'ssh', 'ftp', 'pop3', 'dns', 'ssl', 'snmp', 'irc'}
{'FIN', 'INT', 'CON', 'RST', 'REQ'}
{'FIN', 'ACC', 'INT', 'CON', 'REQ'}
{'INT', 'CON', 'FIN', 'REQ'}
81159
35178


The columns 'id' and 'is_sm_ips_ports' are dropped from the dataframes: 'id' is irrelevent to the data at hand, and 'is_sm_ips_ports' causes errors when calculating z scores (likely because it is always 0).

In [21]:
df_training_set.drop('id', axis = 1, inplace = True)
df_training_set.drop('is_sm_ips_ports', axis = 1, inplace = True)

df_test_set.drop('id', axis = 1, inplace = True)
df_test_set.drop('is_sm_ips_ports', axis = 1, inplace = True)

The numerical data is then normalized via z-score.

In [22]:
#Finds numerical data (columns that are not categorical)
numerical_columns = set(df_training_set.columns.symmetric_difference(categorical_columns))

from scipy.stats import zscore

def z_score_numerical(df, names):
    for name in names:
        df[name] = zscore(df[name])

z_score_numerical(df_training_set, numerical_columns)
z_score_numerical(df_test_set, numerical_columns)

Finally, the next bit of code encodes the now-filtered categorical values into their own columns.

In [23]:
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue) given a dataframe and a list of column names
def encode_text_dummy_loop(df, names):
    for name in names:
        dummies = pd.get_dummies(df[name])
        for x in dummies.columns:
            dummy_name = "{}-{}".format(name, x)
            df[dummy_name] = dummies[x]
        df.drop(name, axis=1, inplace=True)

encode_text_dummy_loop(df_training_set, categorical_columns)
encode_text_dummy_loop(df_test_set, categorical_columns)

The data is now ready for processing.

## Splitting the Data
Split data into training and testing sets.

In [24]:
# Ensure both sets have the same columns after encoding
df_training_set, df_test_set = df_training_set.align(df_test_set, join='inner', axis=1)

# Features (X) and Target (y)
X_train = df_training_set.drop(columns=['label'])
y_train = df_training_set['label']
X_test = df_test_set.drop(columns=['label'])
y_test = df_test_set['label']

# Split training data for validation
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Check data shapes
print(f'Training set shape: {X_train_split.shape}')
print(f'Test set shape: {X_test.shape}')

Training set shape: (60869, 56)
Test set shape: (35178, 56)


## 	Model Training for Fully Connected Neural Network (FCNN)

In [26]:
# Define FCNN Model Function
def create_fcnn(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # For binary classification
    return model

# Create FCNN Model
model_fcnn = create_fcnn((X_train_split.shape[1],))

# Compile model
model_fcnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks for early stopping & model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath=os.path.join(save_path, 'fcnn_best_model.h5'), save_best_only=True)

# Train the model
history_fcnn = model_fcnn.fit(X_train_split, y_train_split, 
                               epochs=50, 
                               batch_size=32, 
                               validation_split=0.2,  # Use 20% of training data for validation
                               callbacks=[early_stopping, model_checkpoint])

# Evaluate the model on the test set
test_loss, test_accuracy = model_fcnn.evaluate(X_test, y_test)
print('FCNN Test Loss: {test_loss}')
print('FCNN Test Accuracy: {test_accuracy}')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

## Model Training for Convolutional Neural Network (CNN)

In [27]:
# Define CNN Model Function
def create_cnn(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # For binary classification
    return model

# Create CNN Model
model_cnn = create_cnn((X_train_split.shape[1],))

# Compile model
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks for early stopping & model checkpoint
early_stopping_cnn = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint_cnn = ModelCheckpoint(filepath=os.path.join(save_path, 'cnn_best_model.h5'), save_best_only=True)

# Train the model
history_cnn = model_cnn.fit(X_train_split, y_train_split, 
                             epochs=50, 
                             batch_size=32, 
                             validation_split=0.2,  # Use 20% of training data for validation
                             callbacks=[early_stopping_cnn, model_checkpoint_cnn])

# Evaluate the model on the test set
test_loss_cnn, test_accuracy_cnn = model_cnn.evaluate(X_test, y_test)
print('CNN Test Loss: {test_loss_cnn}')
print('CNN Test Accuracy: {test_accuracy_cnn}')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

## Evaluate Model Performance (FCNN & CNN)

In [28]:
# For FCNN
y_pred_fcnn = (model_fcnn.predict(X_test) > 0.5).astype("int32")
print("FCNN Classification Report:")
print(classification_report(y_test, y_pred_fcnn))

# For CNN
y_pred_cnn = (model_cnn.predict(X_test) > 0.5).astype("int32")
print("CNN Classification Report:")
print(classification_report(y_test, y_pred_cnn))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).